In [17]:
import numpy as np
import tensorflow as tf

In [4]:
#pip install tensorflow==1.14 --ignore-installed

In [ ]:
#Method one, using LSTM

In [2]:
### reading doc2vec_X file
print('reading files')

vec_num = 100
# fp_train = open('./review_text_train_doc2vec50.csv', 'r')
# fp_test = open('./review_text_test_doc2vec50.csv', 'r')

fp_train = open('./review_text_train_doc2vec100.csv', 'r')
fp_test = open('./review_text_test_doc2vec100.csv', 'r')


reading files


In [3]:
all_sentences = []
for line in fp_train.readlines():
    vectors = [s.strip() for s in line.split(',')]
    all_sentences.append(vectors)
fp_train.close()

all_test_sentences = []
for line in fp_test.readlines():
    vectors = line.split(',')
    all_test_sentences.append(vectors)
fp_test.close()


In [4]:
fp_train_tags = open('review_meta_train.csv', 'r')
all_tags = []
all_votes = []
first_line = True
for line in fp_train_tags.readlines():
    if not first_line:
        votes_tag = line.strip().split(',')[4:]
        votes = votes_tag[0:3]
        tag = votes_tag[-1]
        all_votes.append(votes)
        all_tags.append(tag)
    else:
        first_line = False
fp_train_tags.close()

In [5]:
test_votes = []
fp_test_tags = open('review_meta_test.csv', 'r')
first_line = True
for line in fp_test_tags.readlines():
    if not first_line:
        line_votes = line.strip().split(',')[4:]
        test_votes.append(line_votes)
    else:
        first_line = False
fp_test_tags.close()

In [6]:
for i in range(0, len(all_votes)):
    all_sentences[i] = all_sentences[i][0:vec_num] + all_votes[i]

for i in range(0, len(test_votes)):
    all_test_sentences[i] = all_test_sentences[i][0:vec_num] + test_votes[i]

all_tags_vec = []
for tag in all_tags:
    if tag == '1':
        all_tags_vec.append([1, 0, 0])
    elif tag == '3':
        all_tags_vec.append([0, 1, 0])
    elif tag == '5':
        all_tags_vec.append([0, 0, 1])

In [7]:
all_sentences_embeddings = []
for line in all_sentences:
    line_float = []
    for item in line:
        line_float.append(float(item))
    all_sentences_embeddings.append(line_float)

all_test_sentences_embeddings = []
for line in all_test_sentences:
    line_float = []
    for item in line:
        line_float.append(float(item))
    all_test_sentences_embeddings.append(line_float)

print('file pre-processing finished ...')

file pre-processing finished ...


In [8]:
lr = 0.0005
forget_bias = 1.0
batch_size = 64

n_wvec_dim = 1
n_rows = vec_num + len(all_votes[0])
n_hidden_units = 64
n_classes = 3

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_rows, n_wvec_dim])
y = tf.placeholder(tf.float32, [None, n_classes])

# Define weights
weights = {
    'in': tf.Variable(tf.random_normal([n_wvec_dim, n_hidden_units])),
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}

biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

weights2 = {
    'out': tf.Variable(tf.random_normal([2 * n_hidden_units, n_classes]))
}
biases2 = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}

def biRNN(x, weights, biases):
    x = tf.unstack(x, n_rows, 1)
    lstm_fw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=forget_bias)
    lstm_bw_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=forget_bias)
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_fw_cell, lstm_bw_cell, x, dtype=tf.float32)
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

# LSTM def
def RNN(X, weights, biases):
    X = tf.reshape(X, [-1, n_wvec_dim])
    X_in = tf.matmul(X, weights['in']) + biases['in']
    X_in = tf.reshape(X_in, [-1, n_rows, n_hidden_units])
    ### LSTM 1
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units, forget_bias=1.0)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)
    outputs = tf.unstack(tf.transpose(outputs, [1,0,2]))
    results = tf.matmul(outputs[-1], weights['out']) + biases['out']
    return results

# bi-LSTM
bi_logits = biRNN(x, weights2, biases2)
bi_prediction = tf.nn.softmax(bi_logits)

bi_loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=bi_logits, labels=y))
bi_optimizer = tf.train.AdamOptimizer(learning_rate=lr)
bi_train_op = bi_optimizer.minimize(bi_loss_op)

bi_correct_pred = tf.equal(tf.argmax(bi_prediction, 1), tf.argmax(y, 1))
bi_accuracy = tf.reduce_mean(tf.cast(bi_correct_pred, tf.float32))

# LSTM
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell, unroll=True))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [9]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    max_index = len(all_tags_vec)
    print(max_index)
    start_index = 0
    old_avg_acc = 0
    diff = 100
    iter_cnt = 0
    avg_accuracy = 0
    avg_cnt = 0
    # for i in range(0, training_iters):
    while diff > 0.0001:
        print('iter count: ' + str(iter_cnt))
        step = 0
        finished = False
        avg_accuracy = 0
        avg_cnt = 0
        # start_index = 0
        print('started at index = ' + str(start_index))
        while True:
            batch_xs = []
            batch_ys = []
            if finished:
                break
            if start_index + batch_size < max_index:
                batch_xs = all_sentences_embeddings[start_index: start_index + batch_size]
                batch_ys = all_tags_vec[start_index: start_index + batch_size]
                start_index += batch_size
            else:
                # break
                batch_xs = all_sentences_embeddings[start_index: max_index] + all_sentences_embeddings[0: batch_size - (max_index - start_index)]
                batch_ys = all_tags_vec[start_index: max_index] + all_tags_vec[0: batch_size - (max_index - start_index)]
                start_index = batch_size - (max_index - start_index)
                finished = True

            batch_xs = np.array(batch_xs).reshape([batch_size, n_rows, n_wvec_dim])

            # if not step % 10 == 0:
            ### LSTM
            # sess.run([train_op], feed_dict={
            #     x: batch_xs,
            #     y: batch_ys,
            # })
            ## bi-LSTM
            sess.run(bi_train_op, feed_dict={
                x: batch_xs,
                y: batch_ys,
            })

            if step % 5 == 0:
                ### LSTM
                # temp_acc = sess.run(accuracy, feed_dict={
                #     x: batch_xs,
                #     y: batch_ys,
                # })
                ### bi-LSTM
                temp_acc = sess.run(bi_accuracy, feed_dict={
                    x: batch_xs,
                    y: batch_ys,
                })
                print(temp_acc)
                avg_accuracy += temp_acc
                avg_cnt += 1
            step += 1
        new_avg_acc = avg_accuracy / avg_cnt
        print('new average accuracy = ' + str(new_avg_acc))
        diff = new_avg_acc - old_avg_acc
        old_avg_acc = new_avg_acc
        iter_cnt += 1

    ### predicting
    print('average accuracy = ' + str(avg_accuracy / avg_cnt))
    print('training finished... predicting...')
    finished_pred = False
    start_index_pred = 0
    max_index_pred = len(all_test_sentences_embeddings)
    all_test_pred = []
    while not finished_pred:
        print(start_index_pred)
        if start_index_pred + batch_size < max_index_pred:
            batch_xs_pred = all_test_sentences_embeddings[start_index_pred: start_index_pred + batch_size]
        else:
            # break
            batch_xs_pred = all_test_sentences_embeddings[
                            start_index_pred: max_index_pred] + all_test_sentences_embeddings[
                                                                0: batch_size - (
                                                                            max_index_pred - start_index_pred)]
            finished_pred = True
        start_index_pred += batch_size
        batch_xs_pred = np.array(batch_xs_pred).reshape([batch_size, n_rows, n_wvec_dim])
        ### LSTM
        # preds = sess.run(tf.argmax(pred, 1), feed_dict={x: batch_xs})
        ### bi-LSTM
        preds = sess.run(tf.argmax(bi_prediction, 1), feed_dict={x: batch_xs_pred})
        for item in list(preds):
            if item == 0:
                all_test_pred.append(1)
            elif item == 1:
                all_test_pred.append(3)
            elif item == 2:
                all_test_pred.append(5)
            else:
                all_test_pred.append(0)
    print(len(all_test_pred))
    print(all_test_pred[: len(all_test_sentences_embeddings)])
    ### write to file
    fp_pred = open('./predictions.txt', 'w+')
    res_line = 'Instance_id,rating\n'
    for i in range(0, len(all_test_sentences_embeddings)):
        res_line += str(i + 1) + ',' + str(all_test_pred[i]) + '\n'
    fp_pred.write(res_line)
    fp_pred.close()

28068
iter count: 0
started at index = 0
0.71875
0.65625
0.71875
0.796875
0.71875
0.65625
0.6875
0.703125
0.71875
0.703125
0.703125
0.703125
0.765625
0.703125
0.78125
0.65625
0.65625
0.609375
0.46875
0.5625
0.609375
0.765625
0.578125
0.65625
0.59375
0.765625
0.734375
0.6875
0.640625
0.6875
0.671875
0.8125
0.671875
0.78125
0.765625
0.8125
0.640625
0.8125
0.84375
0.734375
0.71875
0.703125
0.703125
0.765625
0.671875
0.78125
0.625
0.703125
0.609375
0.765625
0.671875
0.6875
0.734375
0.71875
0.6875
0.6875
0.71875
0.640625
0.625
0.6875
0.703125
0.71875
0.671875
0.703125
0.625
0.65625
0.796875
0.734375
0.71875
0.734375
0.6875
0.625
0.78125
0.671875
0.71875
0.65625
0.640625
0.703125
0.546875
0.671875
0.640625
0.734375
0.671875
0.609375
0.75
0.71875
0.671875
0.65625
new average accuracy = 0.6940696022727273
iter count: 1
started at index = 28
0.703125
0.609375
0.8125
0.734375
0.75
0.640625
0.703125
0.703125
0.703125
0.671875
0.65625
0.703125
0.796875
0.6875
0.703125
0.6875
0.75
0.71875
0.515625


In [16]:
#Predictions of Lstm
predictions = open('predictions.txt', 'r')

In [ ]:
# Method 2 ,Using svm to do prediction.

In [18]:
#SVM
### reading doc2vec_X file
print('reading files')
GLOBAL_learning_rate = 0.001
GLOBAL_hidden_units = 128
GLOBAL_layers = 3
GLOBAL_batch_size = 128
GLOBAL_diff_threshold = 0.0001
GLOBAL_forget_bias = 1.0
GLOBAL_vec_num = 50

reading files


In [19]:
fp_train = open('review_text_train_doc2vec200.csv', 'r')
fp_test = open('review_text_test_doc2vec200.csv', 'r')

all_sentences = []
for line in fp_train.readlines():
    vectors = [s.strip() for s in line.split(',')]
    all_sentences.append(vectors)
fp_train.close()

all_test_sentences = []
for line in fp_test.readlines():
    vectors = line.split(',')
    all_test_sentences.append(vectors)
fp_test.close()

fp_train_tags = open('review_meta_train.csv', 'r')
all_tags = []
all_votes = []
first_line = True
for line in fp_train_tags.readlines():
    if not first_line:
        votes_tag = line.strip().split(',')[4:]
        votes = votes_tag[0:3]
        tag = votes_tag[-1]
        all_votes.append(votes)
        all_tags.append(tag)
    else:
        first_line = False
fp_train_tags.close()

In [20]:
test_votes = []
fp_test_tags = open('review_meta_test.csv', 'r')
first_line = True
for line in fp_test_tags.readlines():
    if not first_line:
        line_votes = line.strip().split(',')[4:]
        test_votes.append(line_votes)
    else:
        first_line = False
fp_test_tags.close()

for i in range(0, len(all_votes)):
    all_sentences[i] = all_sentences[i] + all_votes[i]

for i in range(0, len(test_votes)):
    all_test_sentences[i] = all_test_sentences[i] + test_votes[i]

all_tags_vec = []
for tag in all_tags:
    if tag == '1':
        all_tags_vec.append([1, 0, 0])
    elif tag == '3':
        all_tags_vec.append([0, 1, 0])
    elif tag == '5':
        all_tags_vec.append([0, 0, 1])

In [21]:
all_sentences_embeddings = []
for line in all_sentences:
    line_float = []
    for item in line:
        line_float.append(float(item))
    all_sentences_embeddings.append(line_float)


all_test_sentences_embeddings = []
for line in all_test_sentences:
    line_float = []
    for item in line:
        line_float.append(float(item))
    all_test_sentences_embeddings.append(line_float)

In [22]:
### Normalization
from sklearn.preprocessing import normalize as nm

print('normalization')
all_embeddings = list(nm(all_sentences_embeddings + all_test_sentences_embeddings))
all_sentences_embeddings = all_embeddings[: len(all_sentences_embeddings)]
all_test_sentences_embeddings = all_embeddings[len(all_sentences_embeddings): ]

all_tags_num = []
for item in all_tags:
    all_tags_num.append(int(item))
print(all_tags_num)

print('file pre-processing finished ...')

normalization
[1, 5, 5, 3, 3, 5, 5, 3, 5, 3, 5, 1, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 5, 3, 5, 3, 1, 5, 5, 5, 3, 3, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 1, 5, 3, 5, 5, 5, 5, 3, 1, 3, 5, 3, 5, 5, 5, 5, 1, 5, 3, 5, 5, 5, 3, 5, 5, 5, 5, 3, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 3, 3, 3, 5, 1, 1, 5, 5, 5, 5, 5, 5, 5, 1, 3, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 5, 3, 5, 5, 5, 5, 3, 5, 1, 5, 1, 5, 5, 5, 3, 5, 3, 1, 5, 5, 5, 5, 5, 1, 5, 5, 1, 3, 5, 3, 5, 1, 5, 5, 5, 3, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 3, 5, 5, 1, 5, 5, 5, 5, 5, 5, 1, 5, 3, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 3, 5, 5, 5, 3, 3, 5, 5, 5, 1, 3, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5, 3, 5, 5, 5, 5, 5, 3, 3, 5, 5, 5, 5, 5, 5, 5, 1, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 3, 1, 5, 5, 5, 5, 5, 5, 3, 5, 1, 5, 5, 3, 5, 5, 5, 3, 5, 5, 3, 5, 1

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB

In [23]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, LinearSVC

In [25]:
### produce test results
X_train = np.array(all_sentences_embeddings)
y_train = np.array(all_tags_num)
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
model = clf.fit(X_train, y_train)

In [26]:
print('write to file')
y_pred = list(clf.predict(all_test_sentences_embeddings))

fp_pred = open('predictions_svm.txt', 'w+')
# fp_pred = open('../data/res/predictions_linear_svc.txt', 'w+')
res_line = 'Instance_id,rating\n'
for i in range(0, len(y_pred)):
    res_line += str(i + 1) + ',' + str(y_pred[i]) + '\n'
fp_pred.write(res_line)
fp_pred.close()

write to file


In [28]:
#Predictions of Lstm
predictions_svm = open('predictions_svm.txt', 'r')